##Q1 Import and explore the data
Dataset Link: https://drive.google.com/file/d/1tPHOMKD_ef-j8MuynCn2fSpbrL11-J1l/view?usp=sharing


In [ ]:
!ls

pulsar.txt  sample_data


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("pulsar.txt")
df.head(5)

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,121.156250,48.372971,0.375485,-0.013165,3.168896,18.399367,7.449874,65.159298,0.0
1,76.968750,36.175557,0.712898,3.388719,2.399666,17.570997,9.414652,102.722975,0.0
2,130.585938,53.229534,0.133408,-0.297242,2.743311,22.362553,8.508364,74.031324,0.0
3,156.398438,48.865942,-0.215989,-0.171294,17.471572,NaN,2.958066,7.197842,0.0
4,84.804688,36.117659,0.825013,3.274125,2.790134,20.618009,8.405008,76.291128,0.0


In [ ]:
df.isnull().sum()

 Mean of the integrated profile                     0
 Standard deviation of the integrated profile       0
 Excess kurtosis of the integrated profile       1735
 Skewness of the integrated profile                 0
 Mean of the DM-SNR curve                           0
 Standard deviation of the DM-SNR curve          1178
 Excess kurtosis of the DM-SNR curve                0
 Skewness of the DM-SNR curve                     625
target_class                                        0
dtype: int64

In [ ]:
df["target_class"].value_counts()

0.0    11375
1.0     1153
Name: target_class, dtype: int64

##Q2 Carry out data pre-processing

In [ ]:
df.dropna(axis=0,inplace=True)

In [ ]:
df.isnull().sum()

 Mean of the integrated profile                  0
 Standard deviation of the integrated profile    0
 Excess kurtosis of the integrated profile       0
 Skewness of the integrated profile              0
 Mean of the DM-SNR curve                        0
 Standard deviation of the DM-SNR curve          0
 Excess kurtosis of the DM-SNR curve             0
 Skewness of the DM-SNR curve                    0
target_class                                     0
dtype: int64

##Q4 Split the data in traina and test set

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop('target_class', axis=1)
y = df['target_class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=91)

##Q3 Scale the data using Standard Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

####Q3a According to you, why is scaling important in SVM? What are the different methods for scaling the data available in sklearn.preprocessing?



1.   SVM often uses kernel functions (e.g. RBF,sigmoid,linear,poly,etc) that compute distances between data points.
2.   If features are not scaled, the contribution of each feature to the distance calculation may not be equitable.


Different Scalers :
1.   StandardScaler
2.   MinMaxScaler
3.   RobustScaler



##Q5 Implement SVM

Different types of kernels in SVM:
https://www.kaggle.com/discussions/questions-and-answers/326336


###Q5a Linear SVM

Apply Linear SVM for costs (C) = [0.1, 1, 10, 100, 1000], predict for the test set and check f1 score for each of the costs. State the best cost

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score

costs = [0.1, 1, 10, 100, 1000]
f1_scores = {}

for cost in costs:
    svm_model = SVC(kernel='linear', C=cost)
    svm_model.fit(X_train_scaled, y_train)
    y_pred = svm_model.predict(X_test_scaled)
    f1 = f1_score(y_test, y_pred)
    f1_scores[cost] = f1

for cost, f1 in f1_scores.items():
    print(f"F1-Score for C={cost}: {f1}")

F1-Score for C=0.1: 0.8628762541806019
F1-Score for C=1: 0.8618421052631579
F1-Score for C=10: 0.8655737704918033
F1-Score for C=100: 0.8655737704918033
F1-Score for C=1000: 0.8655737704918033


In [ ]:
best_cost = max(f1_scores, key=f1_scores.get)
print("Best Cost:", best_cost)


Best Cost: 10


###Q5b Polynomial Kernel

Apply Linear SVM for degrees = [1,2,3,4,5], predict for the test set and check f1 score for each of the degrees. State the best degree.

In [ ]:
degrees = [1, 2, 3, 4, 5]

f1_scores_poly = {}
for degree in degrees:
    svm_model_poly = SVC(kernel='poly', degree=degree)

    svm_model_poly.fit(X_train_scaled, y_train)
    y_pred_poly = svm_model_poly.predict(X_test_scaled)

    f1_poly = f1_score(y_test, y_pred_poly, average='micro')
    f1_scores_poly[degree] = f1_poly

for degree, f1_poly in f1_scores_poly.items():
    print(f"F1 Score for Degree={degree}: {f1_poly}")

F1 Score for Degree=1: 0.9773584905660377
F1 Score for Degree=2: 0.9741239892183289
F1 Score for Degree=3: 0.9741239892183289
F1 Score for Degree=4: 0.9741239892183289
F1 Score for Degree=5: 0.9725067385444744


In [ ]:
best_degree_poly = max(f1_scores_poly, key=f1_scores_poly.get)
print("Best Degree:", best_degree_poly)

Best Degree: 1


###Q5c Sigmoid Kernel

Apply Linear SVM for costs (C) = [0.1, 1, 10, 100, 1000], predict for the test set and check f1 score for each of the costs. State the best cost

In [ ]:
c = [0.1, 1, 10, 100, 1000]

f1_scores_linear = {}

for cost in c:
    svm_model_linear = SVC(kernel='linear', C=cost)
    svm_model_linear.fit(X_train_scaled, y_train)
    y_pred_linear = svm_model_linear.predict(X_test_scaled)

    f1_linear = f1_score(y_test, y_pred_linear, average='micro')
    f1_scores_linear[cost] = f1_linear

for cost, f1_linear in f1_scores_linear.items():
    print(f"F1 Score for C={cost}: {f1_linear}")

F1 Score for C=0.1: 0.9778975741239893
F1 Score for C=1: 0.9773584905660377
F1 Score for C=10: 0.9778975741239893
F1 Score for C=100: 0.9778975741239893
F1 Score for C=1000: 0.9778975741239893


In [ ]:
best_cost_linear = max(f1_scores_linear, key=f1_scores_linear.get)
print("Best Cost:", best_cost_linear)

Best Cost: 0.1


###Q5d RBF Kernel

Apply Linear SVM for costs (C) = [0.1, 1, 10, 100, 1000], predict for the test set and check f1 score for each of the costs. State the best cost

In [ ]:
c = [0.1,1,10,100,1000]

f1_scores_rbf = {}

for cost in c:
  svm_model_rbf = SVC(kernel='rbf',C=cost)
  svm_model_rbf.fit(X_train_scaled,y_train)
  y_pred_rbf = svm_model_rbf.predict(X_test_scaled)

  f1_rbf = f1_score(y_test,y_pred_rbf)
  f1_scores_rbf[cost] = f1_rbf

for cost,f1_rbf in f1_scores_rbf.items():
    print(f"F1 Score for C = {cost} : {f1_rbf}")

F1 Score for C = 0.1 : 0.8484848484848484
F1 Score for C = 1 : 0.8552631578947367
F1 Score for C = 10 : 0.8701298701298701
F1 Score for C = 100 : 0.8543689320388351
F1 Score for C = 1000 : 0.8535031847133757


In [ ]:
best_cost_rbf = max(f1_scores_rbf, key=f1_scores_rbf.get)
print("Best Cost:", best_cost_rbf)

Best Cost: 10
